# Run: Push for 1M - murine and invertebrate

```
Lead     : ababaian / jt
Issue    : 
Version  : v0.3.3
start    : 2020 06 13
complete : 2020 06 13
files    : ~/serratus/notebook/200612_ab/
s3_files : s3://serratus-public/notebook/200612_ab/
output   : s3://serratus-public/out/200613_1m/
```

### Objectives
- Complete all remaining murine sample (~600K)
- Kick in ~800K invertebrate samples
- Reach 1M accessions / 24 hours
-

## SRA Access - Inveretebrates

Search:
```
("type_rnaseq"[Filter] OR "metagenomic"[Filter] OR "metatranscriptomic"[Filter])  NOT "Vertebrata"[Organism] AND cluster_public[prop] AND "platform illumina"[Properties]
```

Accessed: `2020/06/12`

Results : `2058661` saved as `invert_SraRunInfo.csv`


### Initialize local workspace

In [6]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/200613_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/200613_ab/'

# date and version
date
git rev-parse HEAD # commit version

Sat Jun 13 20:40:10 PDT 2020
010599d09f15d1cc26a9d97eb4f91ac5463a41c0


## SRA Accession Initialization



### Remove previously completed accessions

In [3]:
cd $WORK

# get previous SraRunInfo
aws s3 cp s3://lovelywater2/sra/mu_SraRunInfo.csv.gz ./

gzip -d *.gz

download: s3://lovelywater2/sra/mu_SraRunInfo.csv.gz to ./mu_SraRunInfo.csv.gz


In [4]:
# Create a list of all completed runs to date
cd $WORK
cp ../200612_ab/v2.sra.complete ./
INDEX="v2.sra.complete" # from last notebook

wc -l *.csv

echo " -------------------- "

for SRA in $(ls *.csv)
do
  # Inverse look-up completed SRA
  # into a new SraRunInfo file
  grep -vif $INDEX $SRA > "qc_"$SRA
  
  wc -l $SRA
  wc -l "qc_"$SRA
  echo " -------------------- "
  
done

wc -l qc*csv

   2193741 invert_SraRunInfo.csv
    890747 mu_SraRunInfo.csv
   3084488 total
 -------------------- 
2193741 invert_SraRunInfo.csv
2148749 qc_invert_SraRunInfo.csv
 -------------------- 
890747 mu_SraRunInfo.csv
617567 qc_mu_SraRunInfo.csv
 -------------------- 
   2148749 qc_invert_SraRunInfo.csv
    617567 qc_mu_SraRunInfo.csv
   2766316 total


In [5]:
aws s3 sync ./ $S3_WORK/
# ugh double slash

upload: ./v2.sra.complete to s3://serratus-public/notebook/200613_ab//v2.sra.complete
upload: ./qc_mu_SraRunInfo.csv to s3://serratus-public/notebook/200613_ab//qc_mu_SraRunInfo.csv
upload: ./mu_SraRunInfo.csv to s3://serratus-public/notebook/200613_ab//mu_SraRunInfo.csv
upload: ./qc_invert_SraRunInfo.csv to s3://serratus-public/notebook/200613_ab//qc_invert_SraRunInfo.csv
upload: ./invert_SraRunInfo.csv to s3://serratus-public/notebook/200613_ab//invert_SraRunInfo.csv


### Terraform Initialize

In [9]:
# Terraform customization
git diff $SERRATUS/terraform/main/main.tf

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..d319c00 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "m5.8xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type  

In [24]:
# Initialize terraform
TF=$SERRATUS/terraform/main
cd $TF
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve



Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corresponding provider blocks in configuration, with the constraint strings
suggested below.

* provider.random: version = "~> 2.2"

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
module.download.data.aws_availability_zones.all: Refreshing state...
module.mo

### Terraform initialize 2

In [86]:
# Terraform customization
git diff $SERRATUS/terraform/main/main.tf



diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..f0ee858 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "m5.4xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type

In [97]:
# Initialize terraform -- Round 2
TF=$SERRATUS/terraform/main
cd $TF
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corresponding provider blocks in configuration, with the constraint strings
suggested below.

* provider.random: version = "~> 2.2"

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
module.align.data.aws_ami.amazon_linux_2: Refreshing state...
module.align.data.

### Running Serratus

In [98]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [26]:
# Confirm the upload file -- Mouse
BATCH_SRA=$WORK/qc_mu_SraRunInfo.csv
echo  $BATCH_SRA
wc -l $BATCH_SRA

/home/artem/serratus/notebook/200613_ab/qc_mu_SraRunInfo.csv
617567 /home/artem/serratus/notebook/200613_ab/qc_mu_SraRunInfo.csv


In [28]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $BATCH_SRA

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200613_ab/qc_mu_SraRunInfo.csv
  date: Sat Jun 13 21:54:53 PDT 2020
  wc  : 617567 /home/artem/serratus/notebook/200613_ab/qc_mu_SraRunInfo.csv
  md5 : 8078d7a0cdf500406ef8469a61ea7d99  /home/artem/serratus/notebook/200613_ab/qc_mu_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
74ce70bd0f72ca2c75e1f29e9cb41615  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
5648d6734919fffb6b7c808b7203c9bc  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
d9b3638f254b7042bc88baf22a809534  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
169f453d793b56927b929b88a1d609de  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":

### Run 2

In [100]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [101]:
# Confirm the upload file -- Invertebrates -- Round 2
head -n 1       $WORK/qc_invert_SraRunInfo.csv >> $WORK/qc2_invert_SraRunInfo.csv 
tail -n 1200000 $WORK/qc_invert_SraRunInfo.csv | shuf - >> $WORK/qc2_invert_SraRunInfo.csv 

BATCH_SRA=$WORK/qc2_invert_SraRunInfo.csv

echo  $BATCH_SRA
wc -l $BATCH_SRA

/home/artem/serratus/notebook/200613_ab/qc2_invert_SraRunInfo.csv
1200001 /home/artem/serratus/notebook/200613_ab/qc2_invert_SraRunInfo.csv


In [103]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $BATCH_SRA

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200613_ab/qc2_invert_SraRunInfo.csv
  date: Sun Jun 14 20:02:32 PDT 2020
  wc  : 1200001 /home/artem/serratus/notebook/200613_ab/qc2_invert_SraRunInfo.csv
  md5 : 594dac4e3e203cdd165f6b097bcbef23  /home/artem/serratus/notebook/200613_ab/qc2_invert_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
12a590ac5e04e79808d24c91cd4efc7f  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
29c954de978cbde5b464b264154245ba  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
942ff2f9f0d2eb336789a6c038aa93f3  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
b698f268dc797744915f307e5d95ddc7  tmp.chunk03_sraRunInfo.csv
{

## Run Serratus

In [117]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json
#
cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 50,
  "ALIGN_SCALING_CONSTANT": 0.1,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 0,
  "CLEAR_INTERVAL": 600000,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 15,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 0,
  "GENOME": "cov3ma",
  "MERGE_ARGS": "",
  "MERGE_MAX_INCREASE": 35,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 450,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   539    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":50,"ALIGN_SCALING_CONSTANT":0.1,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":0,"CLEAR_INTERVAL":600000,"DL_ARGS":"","DL_MAX_INCREASE":15

### Logging

max: 1460 4120 90 
cur: 1600 3969 100

```
22:00 0 - - - 
23:00 6000 - - -
00:00 33000 1460 3873 100
01:00 86200 1600 3969 100
09:00 205653 - - -
10:00 370640 soft Reset -----
11:00 382552 1286 760 51
12:00 419540 1010 1940 58
16:45 508042 - - - End of run 137,402
```

### Logging 2
863 before 10AM

```
20:15 0 - - -
21:15 43717 1350 2254 110
22:15 129172 1363 3950 120
23:15 216115 1500 3867 120
07:30 390503 Soft Reset
16:30 828416 done (20:15)
```

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres


#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
#!/bin/bash

for tock in $(seq 1 1000); do
   psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"
   sleep 600 
done

In [ ]:
aws autoscaling describe-auto-scaling-instances --max-items 10 \
  | jq -r '.AutoScalingInstances | .[] | .InstanceId' \
  > asg

# TERMINATE ALL DOWNLOADERS
aws autoscaling describe-auto-scaling-instances \
  | jq -r '.AutoScalingInstances | .[] | select(.AutoScalingGroupName=="serratus-dl-2020061502534816170000000c").InstanceId' \
  > asg2
  
for INSTANCE in $(cat asg2XX); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done


# TERMINATE ALL ALIGNERS
aws autoscaling describe-auto-scaling-instances \
  | jq -r '.AutoScalingInstances | .[] | select(.AutoScalingGroupName=="serratus-align-2020061404510754780000000a").InstanceId' \
  > asg3

for INSTANCE in $(cat asg3XX); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done
  
  
# TERMINATE ALL MERGE
aws autoscaling describe-auto-scaling-instances \
  | jq -r '.AutoScalingInstances | .[] | select(.AutoScalingGroupName=="serratus-merge-2020061404510767000000000c").InstanceId' \
  > asg

for INSTANCE in $(cat asg3XX); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done




In [ ]:
aws autoscaling describe-auto-scaling-instances \
  | jq -r '.AutoScalingInstances | .[] | select(.AutoScalingGroupName=="serratus-align-2020061502534804080000000b").InstanceId' \
  > asg4

split -n 4 asg4

for INSTANCE in $(cat xaa); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done

for INSTANCE in $(cat xab); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done

for INSTANCE in $(cat xac); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done

for INSTANCE in $(cat xad); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done

## Output

Run 2:
Tried again: 828416 SRA in 20.5 hours
16:33
using a 5 minute timer with block deletion did not save
16:34
390,000 SRA in the scheduler stressed out again at like 6 hours in
16:34
I shut-down all instances and did all alignments
16:34
then I did all merge
16:34
both as independent operations
16:35
scheduler became stressed again at ~3000 aligners and 400 merge instances online in the respective phases
16:35
so I dialed it back to ~2200 and 250 to process those steps 'consistently'
16:35
close :slightly_smiling_face: